In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_image='D:/Documents/Capstone/Project -2/Dataset/blast_VS_non-blast/train'

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 2366 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/Dataset/bacterial_leaf_blight_Vs_non-bacterial_leaf_blight/test',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 422 images belonging to 2 classes.


In [5]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
base_model.trainable=False
 
model = tf.keras.Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

In [6]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 batch_normalization_203 (Ba  (None, 5, 5, 1536)       10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

In [7]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [8]:
batch_size=32
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[early])

Epoch 1/10
473/473 [==============================] - 954s 2s/step - loss: 0.3511 - accuracy: 0.8729 - val_loss: 1.4117 - val_accuracy: 0.4833
Epoch 2/10
473/473 [==============================] - 1923s 4s/step - loss: 0.2147 - accuracy: 0.9293 - val_loss: 1.5915 - val_accuracy: 0.5357
Epoch 3/10
473/473 [==============================] - 2112s 4s/step - loss: 0.1784 - accuracy: 0.9428 - val_loss: 2.3655 - val_accuracy: 0.5167
Epoch 4/10
473/473 [==============================] - 2008s 4s/step - loss: 0.1461 - accuracy: 0.9475 - val_loss: 2.3523 - val_accuracy: 0.5190
Epoch 5/10
473/473 [==============================] - 1965s 4s/step - loss: 0.1321 - accuracy: 0.9521 - val_loss: 3.4237 - val_accuracy: 0.4976
Epoch 6/10
473/473 [==============================] - 1672s 4s/step - loss: 0.1155 - accuracy: 0.9615 - val_loss: 2.5604 - val_accuracy: 0.4881
Epoch 7/10
473/473 [==============================] - 1763s 4s/step - loss: 0.1033 - accuracy: 0.9615 - val_loss: 3.0452 - val_accuracy: 